In [1]:
import pandas as pd
from gurobi import *
import matplotlib.pyplot as plt
import math
import numpy as np

In [2]:
booking = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/Booking_new.xlsx")
bd_zones = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/BreakDownZones.xlsx")
dz_to_bd = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/DistanceMatrixDropZoneToBreakDownZone.xlsx")
drop_zones = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/DropZones.xlsx")
bu_zones = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/Outbound.xlsx", sheet_name='BU Zones')
bu_zones_workstations = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/Outbound.xlsx", sheet_name='WorkStation')
bu_zones_to_flight = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/Outbound.xlsx", sheet_name='FlightNumber-BUZone')
flight_default_processing = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/Outbound.xlsx", sheet_name='DefaultProcessingTime')
flight_pre_processing = pd.read_excel("/media/luckshan/New Volume/AirCargo/Ab-Ovo Project/Ab Ovo Data/Outbound.xlsx", sheet_name='Pre-ProcessingBufferTime')

In [3]:
dz = list(drop_zones['Name'].unique())
break_down_zones = list(bd_zones['Name'].unique())

In [4]:
dz_to_bd[['h','m','s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in dz_to_bd['TransportDuration']])

In [5]:
bd_zones[['handling_h','handling_m','handling_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bd_zones['HandlingTimePerULD']])
bd_zones[['transport_WH_h','transport_WH_m','transport_WH_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bd_zones['TransportationTimeToWH']])

In [6]:
booking.columns

Index(['Shipment ID', 'FlightDate', 'ShipmentArrivalDateUTC', 'ProductName',
       'ShipmentOrigin', 'ShipmentDestination', 'CarrierCode', 'Priority',
       'DropZone', 'FlightNumberArrival', 'SerialNumber', 'ArrivalULDNumber',
       'Pieces', 'Weight', 'Volume', 'BookingStatus', 'WeighCMMit',
       'VolumeUnit', 'TotalVolume', 'TotalWeight', 'TotalNumberOfPieces',
       'FlightNumberDeparture', 'ShipmentDepartureDateUTC'],
      dtype='object')

In [7]:
booking['Shipment_arrival_date_time'] = pd.to_datetime(booking['ShipmentArrivalDateUTC'])
booking = booking[booking.ShipmentDepartureDateUTC != '2018-10-24T26:20:00']
booking['Shipment_departure_date_time'] = pd.to_datetime(booking['ShipmentDepartureDateUTC'])
booking['Shipment_duration'] = booking['Shipment_departure_date_time']-booking['Shipment_arrival_date_time']
booking = booking[booking.Shipment_duration>pd.Timedelta(0)]

In [8]:
dz_ship_data = booking[['DropZone','ArrivalULDNumber']]
uld_bd_data = pd.merge(dz_ship_data, dz_to_bd, left_on='DropZone', right_on='DropZoneName')
uld_bd_data['key'] = uld_bd_data['ArrivalULDNumber'].astype(str) + '_' + uld_bd_data['BreakDownZoneName']
uld_bd_data['dz_type'] =  uld_bd_data['DropZone'].apply(lambda x: x.split('-')[0]).str[3:]
uld_bd_data['bd_type1'] = uld_bd_data['BreakDownZoneName'].apply(lambda x: x.split(' ')[1])
uld_bd_data.loc[uld_bd_data['bd_type1'] == 'BD', 'bd_type1'] = "NRML-1"
uld_bd_data['bd_type'] =  uld_bd_data['bd_type1'].apply(lambda x: x.split('-')[0])
uld_bd_data['key_for_type'] = uld_bd_data.apply(lambda x: '1' if x['dz_type'] == x['bd_type'] else '0', axis=1)
uld_bd_data = uld_bd_data[(uld_bd_data.key_for_type == '1') & (uld_bd_data.dz_type == 'NRML')]
uld_bd_data = uld_bd_data.drop_duplicates('key')
uld_bd_data.shape

(1728, 13)

In [9]:
uld_to_bd_times = tupledict(pd.Series(uld_bd_data.m.values,index=[uld_bd_data.ArrivalULDNumber, uld_bd_data.BreakDownZoneName]).to_dict())

In [10]:
booking['build_up_arrival_time'] =  booking['Shipment_arrival_date_time'] #+ np.timedelta64(2, 'h')
booking['build_up_finish_time'] =  booking['Shipment_departure_date_time'] #- np.timedelta64(1, 'h')
booking_new = booking[['ProductName', 'Priority', 'DropZone', 'ArrivalULDNumber', 'Pieces', 'Weight', 'TotalWeight',
         'FlightNumberDeparture', 'Shipment_arrival_date_time', 'Shipment_departure_date_time',
         'Shipment_duration', 'build_up_arrival_time', 'build_up_finish_time']]

In [11]:
booking_new = pd.merge(booking_new, bu_zones_to_flight, left_on='FlightNumberDeparture', right_on='FlightNumber')
booking_new = pd.merge(booking_new, bu_zones, left_on='BU Zone', right_on='Name')

In [12]:
bu_workstation_count = bu_zones_workstations.groupby(['Name']).count().reset_index()
bu_workstation_count.columns = ['Name', 'Count']
bd_zone_all = pd.merge(bu_zones, bu_workstation_count, left_on='Name', right_on='Name')

In [13]:
bd_zone_all = pd.merge(bd_zone_all, bu_zones_to_flight, left_on='Name', right_on='BU Zone')
booking_new = pd.merge(booking_new, bd_zone_all, left_on='FlightNumberDeparture', right_on='FlightNumber')

In [14]:
booking_new[['ULD_handling_h','ULD_handling_m','ULD_handling_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in booking_new['HandlingTimePerULD_x']])

In [15]:
booking_new['arrival_date'] = booking_new['Shipment_arrival_date_time'].dt.normalize()
booking_new['departure_date'] = booking_new['Shipment_departure_date_time'].dt.normalize()

In [16]:
##########################################################
##########################################################
##########################################################

In [57]:
sample_shipments = booking_new.iloc[0:1,:]

In [58]:
sample_shipments['Shipment_duration_minutes'] = (sample_shipments['Shipment_duration'] / np.timedelta64(1, 'm')).astype(int)
sample_shipments['Shipment_arrival_time_minutes'] = pd.to_timedelta(sample_shipments['Shipment_arrival_date_time']).astype('timedelta64[m]').astype(int)

/home/luckshan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/luckshan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  
/home/luckshan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [59]:
bd_zones.head()

,Description,Name,NumberOfWorkstations,SequenceNumber,TransportationTimeToWH,TargetProcessName,HandlingTimePerULD,handling_h,handling_m,handling_s,transport_WH_h,transport_WH_m,transport_WH_s
0,the biggest BD zone for normal ULDs,B BD NRML-1,33,1,00:30:00,NORMAL,00:24:00,0,24,0,0,30,0
1,the 2nd biggest BD zone for normal ULDs,B BD NRML-2,10,2,00:25:00,NORMAL,00:24:00,0,24,0,0,25,0
2,an animal BD zone,BD NML-1,4,3,00:40:00,ANIMAL,00:13:00,0,13,0,0,40,0
3,the biggest BD zone for ULDs that have to be c...,BD CLD-1,8,4,00:30:00,COOLED,00:17:00,0,17,0,0,30,0
4,a normal BD zone,BD NRML-1,5,5,00:30:00,NORMAL,00:20:00,0,20,0,0,30,0


In [60]:
new = sample_shipments[['ArrivalULDNumber', 'Shipment_duration_minutes','Shipment_arrival_time_minutes']]
df_merge= pd.merge(uld_bd_data, new, on='ArrivalULDNumber')
df_merge = df_merge.drop_duplicates('key')
bd_capacity = bd_zones[['Name','NumberOfWorkstations', 'handling_m']]
df_merge1 = df_merge.merge(bd_capacity, left_on='BreakDownZoneName', right_on='Name')

In [61]:
df_merge.shape

(6, 15)

In [62]:
df_merge1['one_hour_duration'] = 60

In [63]:
ArrivalULDNumber_list = list(df_merge1['ArrivalULDNumber'].unique())
BreakDownZoneName_list = list(df_merge1['BreakDownZoneName'].unique())

In [64]:
model = Model('BD zone model')

In [65]:
x = {}
for i in ArrivalULDNumber_list:
    for j in BreakDownZoneName_list:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.ArrivalULDNumber==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range(shipment_arrival_duation_minutes_loop):
            x[i,j,(shipment_arrival_time_minutes_loop+k+1)] = model.addVar(obj=uld_to_bd_times[i, j], vtype=GRB.BINARY)

In [79]:
for i in ArrivalULDNumber_list:
    for j in BreakDownZoneName_list:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.ArrivalULDNumber==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range(shipment_arrival_duation_minutes_loop):
            t = model.addVar(obj=(shipment_arrival_time_minutes_loop+k+1), vtype=GRB.BASIC)

TypeError: 'int' object is not subscriptable

In [51]:
x

{(21231, 'B BD NRML-1', 25671251): <gurobi.Var C0>,
 (21231, 'B BD NRML-1', 25671252): <gurobi.Var C1>,
 (21231, 'B BD NRML-1', 25671253): <gurobi.Var C2>,
 (21231, 'B BD NRML-1', 25671254): <gurobi.Var C3>,
 (21231, 'B BD NRML-1', 25671255): <gurobi.Var C4>,
 (21231, 'B BD NRML-1', 25671256): <gurobi.Var C5>,
 (21231, 'B BD NRML-1', 25671257): <gurobi.Var C6>,
 (21231, 'B BD NRML-1', 25671258): <gurobi.Var C7>,
 (21231, 'B BD NRML-1', 25671259): <gurobi.Var C8>,
 (21231, 'B BD NRML-1', 25671260): <gurobi.Var C9>,
 (21231, 'B BD NRML-1', 25671261): <gurobi.Var C10>,
 (21231, 'B BD NRML-1', 25671262): <gurobi.Var C11>,
 (21231, 'B BD NRML-1', 25671263): <gurobi.Var C12>,
 (21231, 'B BD NRML-1', 25671264): <gurobi.Var C13>,
 (21231, 'B BD NRML-1', 25671265): <gurobi.Var C14>,
 (21231, 'B BD NRML-1', 25671266): <gurobi.Var C15>,
 (21231, 'B BD NRML-1', 25671267): <gurobi.Var C16>,
 (21231, 'B BD NRML-1', 25671268): <gurobi.Var C17>,
 (21231, 'B BD NRML-1', 25671269): <gurobi.Var C18>,
 (2

In [66]:
for i in ArrivalULDNumber_list:
    for j in BreakDownZoneName_list:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.ArrivalULDNumber==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range(shipment_arrival_duation_minutes_loop):
            model.addConstr(quicksum(x[i,b,c] for b in BreakDownZoneName_list for c in range(shipment_arrival_time_minutes_loop+1, shipment_arrival_time_minutes_loop+shipment_arrival_duation_minutes_loop+1)) == 1)

In [67]:
for i in ArrivalULDNumber_list:
    for j in BreakDownZoneName_list:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.ArrivalULDNumber==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        arrival_time_each_uld = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        transport_time_each_bd_zone = list(new_check['m'].unique())[0]
        for k in range(shipment_arrival_duation_minutes_loop):
            model.addConstr(quicksum(x[i,b,c] for b in BreakDownZoneName_list for c in range(shipment_arrival_time_minutes_loop+1, shipment_arrival_time_minutes_loop+shipment_arrival_duation_minutes_loop+1)) >= arrival_time_each_uld+transport_time_each_bd_zone)

In [68]:
for i in ArrivalULDNumber_list:
    for j in BreakDownZoneName_list:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.ArrivalULDNumber==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        handling_minutes = list(new_check['handling_m'].unique())[0]
        cap = list(new_check['NumberOfWorkstations'].unique())[0]
        for k in range(shipment_arrival_duation_minutes_loop):
            model.addConstr(cap - quicksum(x[a,j,c] for a in ArrivalULDNumber_list for c in range(shipment_arrival_time_minutes_loop+1, shipment_arrival_time_minutes_loop+shipment_arrival_duation_minutes_loop+1)) >= quicksum(x[p,j,r] for p in ArrivalULDNumber_list for r in range(shipment_arrival_time_minutes_loop+1, shipment_arrival_time_minutes_loop+handling_minutes+1)))

In [77]:
#x[21231, 'B BD NRML-1', 25671251].x
#df_merge1[df_merge1.ArrivalULDNumber == 76631]

AttributeError: b"Unable to retrieve attribute 'x'"

In [78]:
obj = x.sum()
model.setObjective(obj, GRB.MINIMIZE)

AttributeError: 'dict' object has no attribute 'sum'

In [69]:
model.optimize()

Optimize a model with 1080 rows, 360 columns and 280800 nonzeros
Variable types: 0 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.09 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [70]:
for i in ArrivalULDNumber_list:
    for j in BreakDownZoneName_list:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.ArrivalULDNumber==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range(shipment_arrival_duation_minutes_loop):
            if(x[i,j,(shipment_arrival_time_minutes_loop+k+1)].x>0.5):
                print(i, "assigned to", j, k)# todo

AttributeError: b"Unable to retrieve attribute 'x'"